함수형 프로그래밍은 일급 계층 객체로서의 함수를 강조한다. 함수를 인자로 받거나 함수를 결과 값으로 반환하는 많은 고차 함수를 살펴봤다. 이번 장에서는 함수의 생성과 변경을 돕는 여러 함수가 들어 있는 functools 라이브러리를 살펴본다.

functools 모듈에 있는 함수 중 다음과 같은 것을 살펴본다.

* @lru_cache: 애플리케이션 유형에 따라 이 데코레이터를 사용하면 상당한 성능 향상을 얻을 수 있다.

* @totla_ordering: 이 데코레이터는 다양한 비교 연산자를 만들 수 있도록 돕는다. 이를 통해 객체지향 설계와 함수형 프로그래밍을 어떻게 혼합할 수 있는지에 대한 좀 더 일반적인 질문을 살펴본다.

* partial(): 어떤 기존 함수의 인자 중 일부만을 적용한 새로운 함수를 만든다.

* reduce(): sum()과 같은 축약을 만드는 것을 일반화한 고차 함수다.

### lru_cache로 결과 캐시하기

lru_cache 데코레이터는 주어진 함수를 더 빨리 작동할 수 있는 함수로 바꿔준다. LRU는 최소 최근 사용을 뜻한다. 이러한 방식에서는 최근에 사용한 원소를 풀에 일정하게 유지하다가 풀이 가득찬 경우, 최근에 가장 덜 자주 사용한 원소를 버려 풀이 넘치는 것을 방지한다.

이 함수는 데코레이터다. 따라서 계산 결과를 캐시에 남겨 성능이 향상될 가능성이 있는 모든 함수에 이를 적용할 수 있다. 사용법은 다음과 같다.

In [1]:
from functools import lru_cache
@lru_cache(128)
def fibc(n):
    if n == 0: return 0
    if n == 1: return 1
    return fibc(n-1) + fibc(n-2)

캐시에 n이 있다면 비용이 많이 들 수도 있는 계산을 다시 하지 않고, 캐시에 있는 예전에 계산했던 결과를 반환한다. 각 반환 값은 캐시에 추가된다. 캐시가 가득차면, 가장 오래된 값을 없애 새로운 값이 들어갈 공간을 만든다.

재귀를 조심성 없이 구현하면, 비용이 꽤 많이 든다. 전제 복잡도는 O(n^2)이다.

In [2]:
fibc(20)

6765

In [3]:
fibc(200)

280571172992510140037611932413038677189525

In [4]:
fibc(1000)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

In [13]:
fibc(5000)

3878968454388325633701916308325905312082127714646245106160597214895550139044037097010822916462210669479293452858882973813483102008954982940361430156911478938364216563944106910214505634133706558656238254656700712525929903854933813928836378347518908762970712033337052923107693008518093849801803847813996748881765554653788291644268912980384613778969021502293082475666346224923071883324803280375039130352903304505842701147635242270210934637699104006714174883298422891491273104054328753298044273676822977244987749874555691907703880637046832794811358973739993110106219308149018570815397854379195305617510761053075688783766033667355445258844886241619210553457493675897849027988234351023599844663934853256411952221859563060475364645470760330902420806382584929156452876291575759142343809142302917491088984155209854432486594079793571316841692868039545309545388698114665082066862897420639323438488465240988742395873801976993820317174208932265468879364002630797780058759129671389634214252579116872755600360311370

이렇게 메모이제이션이라는 개념은 강력하다. 결과를 메모이제이션하면 성능이 향상되는 알고리즘도 많다. 하지만 일부 알고리즘은 그리 큰 이륵을 보지 못하기도 한다. 

비슷한 값을 여러 번 계산하는 프로그램이라면 성능을 향상시킬 수있다. 하지만 캐시에 넣은 값을 재활용하는 일이 적다면, 캐시를 유지하는 데 드는 비용이 성능 향상보다 더 커져 버린다.

이항 계수를 lru_cache로 다루면 캐시를 초기화하는 연산에 따른 비용이 더 들어가 실제보다 성능이 더 나쁘게 나타난다. lru_cache 데코레이터를 추가하는 것은 매우 쉽지만, 그 결과는 심오할 수도 있고 실제 데이터의 분포화 알고리즘에 따라서는 아무런 영향이 없을 수도 있다는 것이다. 

따라서 캐시가 상태가 있는 객체라는 사실을 알아두는 것이 중요하다. 이 설계는 순수 함수형 프로그래밍의 한계를 넘어선다. 이상적인 것은 대입문을 없애고 그에 따른 상태 변화를 피하는 것이다. 이러한 상태 변화를 피하는 것의 전형적인 예가 바로 재귀함수다. 재귀함수에서는 현재 상태가 함수의 인자 값에 들어있지, 어떤 변수에 들어 있지 않다. 하지만 이렇게 이상화한 재귀가 실제 프로세서 하드웨어와 메모리에 제약하에서 효율적으로 작동하려면 꼬리재귀호출 회적화가 필수적이라는 것을 살펴봤다. 파이썬에서는 꼬리재귀를 for 루프로 바꿔 수동으로 최적화할 수 있다. 캐시도 필요할 때마다 수동으로 도입할 수 잇는 비슷한 종류의 최적화다.

### 완전한 순서가 정해져 있는 클래스 정의하기

